
# **Laboratorio de aprendizaje por refuerzo**

## Introducción

El aprendizaje por refuerzo es un enfoque general para aprender una política de control en sistemas dinámicos estocásticos a través de la interacción. En este paradigma, un agente aprende a optimizar su comportamiento acumulando recompensas con el tiempo. A diferencia de la optimización local, el aprendizaje por refuerzo se enfoca en descubrir el control óptimo incluso cuando las recompensas están diferidas en el tiempo.

En este laboratorio, exploraremos dos algoritmos clave:
1. **SARSA** (*State-Action-Reward-State-Action*): Un algoritmo *on-policy* que actualiza los valores de las acciones en función de la política que sigue el agente.
2. **Q-Learning**: Un algoritmo *off-policy* que estima el valor de la política óptima independientemente de la política actual del agente.

Además, abordaremos el dilema de **exploración-explotación**, donde el agente debe balancear entre:
- **Explorar**: Probar nuevas acciones para descubrir más sobre el entorno.
- **Explotar**: Utilizar lo aprendido para maximizar las recompensas conocidas.

Para simplificar el aprendizaje de los algoritmos, utilizaremos el entorno **FrozenLake-v0** de OpenAI Gym, un problema sencillo que permite ilustrar los conceptos básicos del aprendizaje por refuerzo.


## Marco Teórico

### **Elementos del Aprendizaje por Refuerzo**
1. **Agente**: Entidad que toma decisiones.
2. **Entorno**: Sistema en el que opera el agente.
3. **Estados ($ S $)**: Configuraciones posibles del entorno.
4. **Acciones ($ A $)**: Conjunto de decisiones posibles para el agente.
5. **Recompensas ($ R $)**: Retroalimentación numérica que guía al agente.
6. **Política ($ \pi $)**: Estrategia que sigue el agente para tomar decisiones.

### **Elementos del Aprendizaje por Refuerzo en el Contexto de FrozenLake**

#### 1. **Agente**:
- **Definición**: El agente es quien toma decisiones en el entorno para maximizar la recompensa acumulada.
- **En FrozenLake**: 
  - El agente es el jugador que debe moverse desde la posición inicial `S` (Start) hasta la meta `G` (Goal) evitando caer en los agujeros `H`.

---

#### 2. **Entorno**:
- **Definición**: El sistema en el que opera el agente. Proporciona estados y recompensas en respuesta a las acciones tomadas por el agente.
- **En FrozenLake**:
  - El entorno es el mapa representado como una cuadrícula:
    - `S`: Inicio.
    - `F`: Hielo congelado (camino seguro).
    - `H`: Agujeros (estado terminal negativo).
    - `G`: Meta (estado terminal positivo).

---

#### 3. **Estados ($ S $)**:
- **Definición**: Representan las configuraciones posibles del entorno en un momento dado.
- **En FrozenLake**:
  - Los estados corresponden a las posiciones del agente en la cuadrícula.
  - Por ejemplo, en un mapa de 4x4, hay $ 16 $ estados posibles numerados de $ 0 $ a $ 15 $.
  - El estado se calcula como:
    \[
    \text{Estado} = \text{fila} \times \text{número de columnas} + \text{columna}
    \]

---

#### 4. **Acciones ($ A $)**:
- **Definición**: Conjunto de decisiones que el agente puede tomar desde un estado dado.
- **En FrozenLake**:
  - El agente tiene 4 acciones disponibles:
    - `0`: Izquierda.
    - `1`: Abajo.
    - `2`: Derecha.
    - `3`: Arriba.
  - En modo resbaladizo (`is_slippery=True`), la acción tomada puede no coincidir con el movimiento efectivo debido a la estocasticidad.

---

#### 5. **Recompensas ($ R $)**:
- **Definición**: Retroalimentación numérica que guía al agente hacia su objetivo.
- **En FrozenLake**:
  - Las recompensas están definidas como:
    - **Meta (`G`)**: $ +1 $.
    - **Hielo (`F`) o Inicio (`S`)**: $ 0 $.
    - **Agujero (`H`)**: $ 0 $.

---

#### 6. **Política ($ \pi $)**:
- **Definición**: Estrategia que sigue el agente para tomar decisiones en cada estado.
- **En FrozenLake**:
  - La política puede ser:
    - **Inicial**: Selección aleatoria de acciones (exploración).
    - **Aprendida**: Después de entrenar al agente, la política selecciona la acción con el mayor valor $ Q(s, a) $ en cada estado.


### **Algoritmos de Aprendizaje**

1. **SARSA**:
   - SARSA es un algoritmo *on-policy*, lo que significa que actualiza los valores $ Q(s, a) $ basándose en la política actual.
   - Ecuación de actualización:
     $$
     Q_{t+1}(s_t, a_t) \leftarrow Q_t(s_t, a_t) + \alpha \left[ r_t + \gamma Q_t(s_{t+1}, a_{t+1}) - Q_t(s_t, a_t) \right]
     $$
     Donde:
     - $ \alpha $: Tasa de aprendizaje.
     - $ \gamma $: Factor de descuento.
     - $ r_t $: Recompensa en el paso $ t $.

2. **Q-Learning**:
   - Q-Learning es un algoritmo *off-policy* que busca la política óptima sin seguir necesariamente la política actual.
   - Ecuación de actualización:
     $$
     Q_{t+1}(s_t, a_t) \leftarrow Q_t(s_t, a_t) + \alpha \left[ r_t + \gamma \max_b Q_t(s_{t+1}, b) - Q_t(s_t, a_t) \right]
     $$


### **Estrategias de Exploración**
1. **ε-greedy**:
   - Con probabilidad $ 1-\epsilon $, el agente elige la mejor acción conocida.
   - Con probabilidad $ \epsilon $, selecciona una acción aleatoria para explorar.
2. **Softmax**:
   - Asigna probabilidades a cada acción según sus valores $ Q(s, a) $:
     $$
     P(a_i|s) = \frac{e^{\frac{Q(s, a_i)}{\tau}}}{\sum_j e^{\frac{Q(s, a_j)}{\tau}}}
     $$
     Donde $ \tau $ controla el nivel de exploración.

### **Evaluación y Rendimiento**
1. Para evaluar el aprendizaje, es esencial contar los éxitos en ventanas de episodios (por ejemplo, 100).
2. Las políticas aprendidas deben evaluarse repetidamente para obtener un promedio confiable, especialmente en entornos estocásticos.


In [1]:
import random

class FrozenLake:
    def __init__(self, map_desc=None, is_slippery=True):
        self.map_desc = map_desc or [
            "SFFF",
            "FHFH",
            "FFFH",
            "HFFG"
        ]
        self.is_slippery = is_slippery
        self.n_rows = len(self.map_desc)
        self.n_cols = len(self.map_desc[0])
        self.start_state = self._find_start()
        self.state = self.start_state
        self.terminated = False

        # Diccionario de movimientos (acciones) corregido
        self.actions = {
            0: (0, -1),  # Izquierda
            1: (1, 0),   # Abajo
            2: (0, 1),   # Derecha
            3: (-1, 0)   # Arriba
        }

        # Diccionario para acciones perpendiculares
        self.perpendicular_actions = {
            0: [3, 1],  # Izquierda: Arriba y Abajo
            1: [0, 2],  # Abajo: Izquierda y Derecha
            2: [1, 3],  # Derecha: Abajo y Arriba
            3: [0, 2]   # Arriba: Izquierda y Derecha
        }

    def _find_start(self):
        """Encuentra la posición inicial (S) en el mapa."""
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.map_desc[row][col] == "S":
                    return (row, col)
        raise ValueError("Mapa no contiene estado inicial 'S'.")

    def reset(self):
        """Reinicia el juego al estado inicial."""
        self.state = self.start_state
        self.terminated = False
        return self.state

    def _inside_bounds(self, row, col):
        """Verifica si la posición está dentro del mapa."""
        return 0 <= row < self.n_rows and 0 <= col < self.n_cols

    def is_hole(self, state):
        """Verifica si un estado es un agujero."""
        row, col = state
        return self.map_desc[row][col] == "H"

    def is_goal(self, state):
        """Verifica si un estado es el objetivo."""
        row, col = state
        return self.map_desc[row][col] == "G"

    def step(self, action):
        """Aplica una acción y devuelve el nuevo estado, recompensa, y si terminó."""
        if self.terminated:
            raise ValueError("El episodio ya terminó. Reinicia el juego con reset().")

        # Aplicar estocasticidad si is_slippery=True
        if self.is_slippery:
            probabilities = [1/3, 1/3, 1/3]
            actions = [action] + self.perpendicular_actions[action]
            action = random.choices(actions, probabilities)[0]

        # Determinar la nueva posición
        move = self.actions[action]
        new_row = self.state[0] + move[0]
        new_col = self.state[1] + move[1]

        # Si la nueva posición está fuera de los límites, el agente no se mueve
        if not self._inside_bounds(new_row, new_col):
            new_row, new_col = self.state

        # Actualizar estado
        new_state = (new_row, new_col)
        tile = self.map_desc[new_row][new_col]

        # Determinar recompensa y si terminó el episodio
        reward = -0.1  # Penalización por paso
        if tile == "H":  # Agujero
            reward = -1
            self.terminated = True
        elif tile == "G":  # Meta
            reward = 1
            self.terminated = True

        self.state = new_state
        return new_state, reward, self.terminated

    def render(self):
        """Muestra el mapa actual con la posición del agente."""
        for row in range(self.n_rows):
            row_str = ""
            for col in range(self.n_cols):
                if (row, col) == self.state:
                    row_str += "A"  # Agente
                else:
                    row_str += self.map_desc[row][col]
            print(row_str)
        print()


## **Ejercicios**

### **Ejercicio 1: Familiarización con OpenAI Gym**
Este ejercicio tiene como propósito entender cómo interactuar con el entorno FrozenLake-v1 y explorar sus características principales. Esto incluye:
- **Objetivo**:
  - Familiarizarse con los espacios de observación y acción.
  - Observar cómo las acciones afectan el estado y la recompensa.
  - Experimentar con los parámetros clave, como:
    - Mapas personalizados.
    - Naturaleza resbaladiza del entorno.

In [ ]:
import random

class FrozenLake:
    def __init__(self, map_desc=None, is_slippery=True):
        self.map_desc = map_desc or [
            "SFFF",
            "FHFH",
            "FFFH",
            "HFFG"
        ]
        self.is_slippery = is_slippery
        self.n_rows = len(self.map_desc)
        self.n_cols = len(self.map_desc[0])
        self.start_state = self._find_start()
        self.state = self.start_state
        self.terminated = False

        # Diccionario de movimientos (acciones) corregido
        self.actions = {
            0: (0, -1),  # Izquierda
            1: (1, 0),   # Abajo
            2: (0, 1),   # Derecha
            3: (-1, 0)   # Arriba
        }

        # Diccionario para acciones perpendiculares
        self.perpendicular_actions = {
            0: [3, 1],  # Izquierda: Arriba y Abajo
            1: [0, 2],  # Abajo: Izquierda y Derecha
            2: [1, 3],  # Derecha: Abajo y Arriba
            3: [0, 2]   # Arriba: Izquierda y Derecha
        }

    def _find_start(self):
        """Encuentra la posición inicial (S) en el mapa."""
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.map_desc[row][col] == "S":
                    return (row, col)
        raise ValueError("Mapa no contiene estado inicial 'S'.")

    def reset(self):
        """Reinicia el juego al estado inicial."""
        self.state = self.start_state
        self.terminated = False
        return self.state

    def _inside_bounds(self, row, col):
        """Verifica si la posición está dentro del mapa."""
        return 0 <= row < self.n_rows and 0 <= col < self.n_cols

    def is_hole(self, state):
        """Verifica si un estado es un agujero."""
        row, col = state
        return self.map_desc[row][col] == "H"

    def is_goal(self, state):
        """Verifica si un estado es el objetivo."""
        row, col = state
        return self.map_desc[row][col] == "G"

    def step(self, action):
        """Aplica una acción y devuelve el nuevo estado, recompensa, y si terminó."""
        if self.terminated:
            raise ValueError("El episodio ya terminó. Reinicia el juego con reset().")

        # Aplicar estocasticidad si is_slippery=True
        if self.is_slippery:
            probabilities = [1/3, 1/3, 1/3]
            actions = [action] + self.perpendicular_actions[action]
            action = random.choices(actions, probabilities)[0]

        # Determinar la nueva posición
        move = self.actions[action]
        new_row = self.state[0] + move[0]
        new_col = self.state[1] + move[1]

        # Si la nueva posición está fuera de los límites, el agente no se mueve
        if not self._inside_bounds(new_row, new_col):
            new_row, new_col = self.state

        # Actualizar estado
        new_state = (new_row, new_col)
        tile = self.map_desc[new_row][new_col]

        # Determinar recompensa y si terminó el episodio
        reward = -0.1  # Penalización por paso
        if tile == "H":  # Agujero
            reward = -1
            self.terminated = True
        elif tile == "G":  # Meta
            reward = 1
            self.terminated = True

        self.state = new_state
        return new_state, reward, self.terminated

    def render(self):
        """Muestra el mapa actual con la posición del agente."""
        for row in range(self.n_rows):
            row_str = ""
            for col in range(self.n_cols):
                if (row, col) == self.state:
                    row_str += "A"  # Agente
                else:
                    row_str += self.map_desc[row][col]
            print(row_str)
        print()



### **Ejercicio 2: SARSA**
- **Objetivo**:
  - Implementar el algoritmo SARSA con exploración **ε-greedy**.
  - Actualizar una tabla $ Q $ de valores para cada estado y acción.
  - Medir el rendimiento contando los episodios exitosos en ventanas de 100 episodios.
  - Considerar el problema resuelto si la tasa de éxito supera el 76%.

In [128]:
import numpy as np
import random

class FrozenLake:
    def __init__(self, map_desc=None, is_slippery=True):
        self.map_desc = map_desc or [
            "SFFF",
            "FHFH",
            "FFFH",
            "HFFG"
        ]
        self.is_slippery = is_slippery
        self.n_rows = len(self.map_desc)
        self.n_cols = len(self.map_desc[0])
        self.start_state = self._find_start()
        self.state = self.start_state
        self.terminated = False

        self.actions = {
            0: (0, -1),  # Izquierda
            1: (1, 0),   # Abajo
            2: (0, 1),   # Derecha
            3: (-1, 0)   # Arriba
        }

        self.perpendicular_actions = {
            0: [3, 1],  # Izquierda: Arriba y Abajo
            1: [0, 2],  # Abajo: Izquierda y Derecha
            2: [1, 3],  # Derecha: Abajo y Arriba
            3: [0, 2]   # Arriba: Izquierda y Derecha
        }

    def _find_start(self):
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.map_desc[row][col] == "S":
                    return (row, col)
        raise ValueError("El mapa no contiene un estado inicial 'S'.")

    def reset(self):
        self.state = self.start_state
        self.terminated = False
        return self.state

    def _inside_bounds(self, row, col):
        return 0 <= row < self.n_rows and 0 <= col < self.n_cols

    def step(self, action):
        if self.terminated:
            raise ValueError("El episodio ya terminó. Reinicia el entorno.")

        if self.is_slippery:
            probabilities = [1/3, 1/3, 1/3]
            actions = [action] + self.perpendicular_actions[action]
            action = random.choices(actions, probabilities)[0]

        move = self.actions[action]
        new_row = self.state[0] + move[0]
        new_col = self.state[1] + move[1]

        if not self._inside_bounds(new_row, new_col):
            new_row, new_col = self.state

        new_state = (new_row, new_col)
        tile = self.map_desc[new_row][new_col]

        reward = 0
        if tile == "H":  # Agujero
            self.terminated = True
        elif tile == "G":  # Meta
            reward = 1
            self.terminated = True

        self.state = new_state
        return new_state, reward, self.terminated

def state_to_index(state, n_cols):
    return state[0] * n_cols + state[1]

def epsilon_greedy(q_table, state_index, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice([0, 1, 2, 3])  # Exploración
    else:
        return np.argmax(q_table[state_index])  # Explotación

def train_sarsa(lake, episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.5, epsilon_decay=0.995):
    n_states = lake.n_rows * lake.n_cols
    n_actions = 4
    q_table = np.zeros((n_states, n_actions))
    successes = 0

    for episode in range(episodes):
        state = lake.reset()
        state_index = state_to_index(state, lake.n_cols)
        action = epsilon_greedy(q_table, state_index, epsilon)
        total_reward = 0
        done = False

        while not done:
            next_state, reward, done = lake.step(action)
            next_state_index = state_to_index(next_state, lake.n_cols)
            next_action = epsilon_greedy(q_table, next_state_index, epsilon)

            # Actualización SARSA
            q_table[state_index, action] += alpha * (
                reward + gamma * q_table[next_state_index, next_action] - q_table[state_index, action]
            )

            state_index = next_state_index
            action = next_action
            total_reward += reward

        if reward > 0:
            successes += 1

        epsilon = max(0.01, epsilon * epsilon_decay)  # Decaimiento de ε

        # Información de progreso
        if (episode + 1) % 100 == 0:
            print(f"Episodio {episode + 1}/{episodes}: Tasa de éxito: {successes / (episode + 1):.2%}")

    print("\nEntrenamiento completado.")
    print(f"Tasa de éxito final: {successes / episodes:.2%}")
    return q_table

def evaluate_policy(lake, q_table, episodes=100):
    successes = 0
    for _ in range(episodes):
        state = lake.reset()
        done = False
        while not done:
            state_index = state_to_index(state, lake.n_cols)
            action = np.argmax(q_table[state_index])  # Política Greedy
            state, reward, done = lake.step(action)

        if reward > 0:
            successes += 1

    success_rate = successes / episodes
    print(f"\nEvaluación completada. Tasa de éxito: {success_rate:.2%}")
    return success_rate

# Crear el entorno
lake = FrozenLake(is_slippery=True)

# Entrenar SARSA
q_table = train_sarsa(lake, episodes=3000)

# Evaluar la política aprendida
evaluate_policy(lake, q_table, episodes=100)


Episodio 100/3000: Tasa de éxito: 2.00%
Episodio 200/3000: Tasa de éxito: 7.50%
Episodio 300/3000: Tasa de éxito: 9.67%
Episodio 400/3000: Tasa de éxito: 15.00%
Episodio 500/3000: Tasa de éxito: 20.80%
Episodio 600/3000: Tasa de éxito: 24.17%
Episodio 700/3000: Tasa de éxito: 29.57%
Episodio 800/3000: Tasa de éxito: 33.12%
Episodio 900/3000: Tasa de éxito: 36.11%
Episodio 1000/3000: Tasa de éxito: 38.00%
Episodio 1100/3000: Tasa de éxito: 40.00%
Episodio 1200/3000: Tasa de éxito: 40.67%
Episodio 1300/3000: Tasa de éxito: 41.31%
Episodio 1400/3000: Tasa de éxito: 42.43%
Episodio 1500/3000: Tasa de éxito: 42.67%
Episodio 1600/3000: Tasa de éxito: 43.44%
Episodio 1700/3000: Tasa de éxito: 44.18%
Episodio 1800/3000: Tasa de éxito: 44.50%
Episodio 1900/3000: Tasa de éxito: 45.84%
Episodio 2000/3000: Tasa de éxito: 46.80%
Episodio 2100/3000: Tasa de éxito: 47.71%
Episodio 2200/3000: Tasa de éxito: 48.50%
Episodio 2300/3000: Tasa de éxito: 49.17%
Episodio 2400/3000: Tasa de éxito: 49.83%
Epis

0.73

### **Conclusión sobre los Resultados**

La evaluación del agente muestra una **tasa de éxito del 73%**, mientras que la **tasa final durante el entrenamiento** fue del 51.67%. Este comportamiento destaca lo siguiente:

1. **Desempeño del Agente en Entrenamiento**:
   - Durante el entrenamiento, el agente sigue una estrategia $ \epsilon $-greedy que prioriza la exploración. Esto introduce decisiones aleatorias que reducen la tasa de éxito final del entrenamiento, dejándola en 51.67%.

2. **Desempeño del Agente en Evaluación**:
   - En la evaluación, el agente deja de explorar ($ \epsilon = 0 $) y sigue exclusivamente la política aprendida. Esto mejora significativamente su desempeño, alcanzando una tasa de éxito del 73%.

3. **Calidad de la Política Aprendida**:
   - El agente ha aprendido una política que logra buenos resultados en un entorno desafiante. Sin embargo, la diferencia entre el 51.67% y el 73% indica que aún hay margen para optimizar el proceso de aprendizaje.

4. **Dirección para Mejoras**:
   - Incrementar el número de episodios de entrenamiento.
   - Ajustar los parámetros $ \alpha $ (tasa de aprendizaje) y $ \epsilon $ (exploración) para lograr una convergencia más rápida.


### **Ejercicio 3: Q-Learning**
- **Objetivo**:
  - Sustituir la regla de actualización de SARSA por la de Q-Learning.
  - Comparar el rendimiento entre SARSA y Q-Learning para analizar sus diferencias.


In [3]:
import numpy as np
import random

class QLearning:
    def __init__(self, lake, episodes=5000, alpha=0.7, gamma=0.9, epsilon=0.1):
        self.lake = lake
        self.episodes = episodes
        self.alpha = alpha  # Tasa de aprendizaje
        self.gamma = gamma  # Factor de descuento
        self.epsilon = epsilon  # Exploración inicial
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Inicializar tabla Q

    def epsilon_greedy(self, state_index):
        """Selecciona una acción usando el método ε-greedy."""
        if np.random.uniform(0, 1) < self.epsilon:
            return random.choice([0, 1, 2, 3])  # Exploración
        else:
            return np.argmax(self.q_table[state_index])  # Explotación

    def train(self):
        """Entrena el agente usando Q-Learning."""
        success_count = 0  # Contador de éxitos
        success_history = []  # Historial de tasa de éxito

        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False

            while not done:
                # Seleccionar acción usando ε-greedy
                action = self.epsilon_greedy(state_index)

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Actualizar tabla Q
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                # Actualizar estado
                state_index = next_state_index
                done = terminated

                # Contar éxito si se alcanza la meta
                if reward > 0:
                    success_count += 1

            # Reducir \( \epsilon \) gradualmente
            self.epsilon = max(0.01, self.epsilon * 0.995)

            # Guardar la tasa de éxito cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = (success_count / (episode + 1)) * 100
                success_history.append(success_rate)
                print(f"Episodio {episode + 1}/{self.episodes}: Tasa de éxito: {success_rate:.2f}%")

        print("Entrenamiento completado.")
        print(f"Tasa de éxito final: {success_rate:.2f}%")
        return success_history

    def evaluate(self, evaluation_episodes=100):
        """Evalúa el agente después del entrenamiento."""
        success_count = 0

        for _ in range(evaluation_episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False

            while not done:
                # Seguir la mejor acción según la tabla Q
                action = np.argmax(self.q_table[state_index])
                next_state, reward, terminated = self.lake.step(action)
                state_index = next_state[0] * self.lake.n_cols + next_state[1]
                done = terminated

                if reward > 0:
                    success_count += 1

        success_rate = (success_count / evaluation_episodes) * 100
        print(f"Evaluación completada. Tasa de éxito: {success_rate:.2f}%")
        return success_rate


# Crear el entorno FrozenLake
class FrozenLake:
    def __init__(self, map_desc=None, is_slippery=True):
        self.map_desc = map_desc or [
            "SFFF",
            "FHFH",
            "FFFH",
            "HFFG"
        ]
        self.is_slippery = is_slippery
        self.n_rows = len(self.map_desc)
        self.n_cols = len(self.map_desc[0])
        self.start_state = self._find_start()
        self.state = self.start_state
        self.terminated = False
        self.actions = {
            0: (0, -1),  # Izquierda
            1: (1, 0),   # Abajo
            2: (0, 1),   # Derecha
            3: (-1, 0)   # Arriba
        }
        self.perpendicular_actions = {
            0: [3, 1],
            1: [0, 2],
            2: [1, 3],
            3: [0, 2]
        }

    def _find_start(self):
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.map_desc[row][col] == "S":
                    return (row, col)
        raise ValueError("Mapa no contiene estado inicial 'S'.")

    def reset(self):
        self.state = self.start_state
        self.terminated = False
        return self.state

    def step(self, action):
        if self.terminated:
            raise ValueError("El episodio ya terminó. Reinicia el juego con reset().")

        if self.is_slippery:
            probabilities = [1/3, 1/3, 1/3]
            actions = [action] + self.perpendicular_actions[action]
            action = random.choices(actions, probabilities)[0]

        move = self.actions[action]
        new_row = self.state[0] + move[0]
        new_col = self.state[1] + move[1]

        if not (0 <= new_row < self.n_rows and 0 <= new_col < self.n_cols):
            new_row, new_col = self.state

        self.state = (new_row, new_col)
        tile = self.map_desc[new_row][new_col]

        if tile == "H":
            self.terminated = True
            return self.state, -1, True
        elif tile == "G":
            self.terminated = True
            return self.state, 1, True
        else:
            return self.state, -0.1, False


# Configurar el entorno y entrenar
lake = FrozenLake(is_slippery=True)
agent = QLearning(lake, episodes=5000)
agent.train()
agent.evaluate()


Episodio 100/5000: Tasa de éxito: 16.00%
Episodio 200/5000: Tasa de éxito: 21.00%
Episodio 300/5000: Tasa de éxito: 23.33%
Episodio 400/5000: Tasa de éxito: 27.25%
Episodio 500/5000: Tasa de éxito: 31.20%
Episodio 600/5000: Tasa de éxito: 31.67%
Episodio 700/5000: Tasa de éxito: 32.86%
Episodio 800/5000: Tasa de éxito: 33.88%
Episodio 900/5000: Tasa de éxito: 33.67%
Episodio 1000/5000: Tasa de éxito: 33.40%
Episodio 1100/5000: Tasa de éxito: 34.09%
Episodio 1200/5000: Tasa de éxito: 35.25%
Episodio 1300/5000: Tasa de éxito: 35.62%
Episodio 1400/5000: Tasa de éxito: 36.29%
Episodio 1500/5000: Tasa de éxito: 37.00%
Episodio 1600/5000: Tasa de éxito: 37.25%
Episodio 1700/5000: Tasa de éxito: 37.71%
Episodio 1800/5000: Tasa de éxito: 38.22%
Episodio 1900/5000: Tasa de éxito: 38.68%
Episodio 2000/5000: Tasa de éxito: 39.20%
Episodio 2100/5000: Tasa de éxito: 39.81%
Episodio 2200/5000: Tasa de éxito: 40.55%
Episodio 2300/5000: Tasa de éxito: 41.39%
Episodio 2400/5000: Tasa de éxito: 41.54%
E

74.0

### **Ejercicio 4: Exploración Softmax**
- **Objetivo**:
  - Implementar una estrategia de exploración Softmax.
  - Asignar probabilidades de selección de acciones proporcionalmente a sus valores $ Q(s, a) $.

In [4]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import random


class SoftmaxQLearningVisualizer:
    def __init__(self, root, lake, episodes=500, tau=1.0):
        self.root = root
        self.lake = lake
        self.episodes = episodes
        self.tau = tau  # Temperatura inicial
        self.alpha = 0.5  # Tasa de aprendizaje
        self.gamma = 0.99  # Factor de descuento
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Tabla Q

        # Configuración del episodio actual
        self.current_episode = 0
        self.state = self.lake.reset()
        self.done = False
        self.success_count = 0
        self.success_history = []  # Historial de éxitos (en ventanas de 100 episodios)

        # Configuración de la interfaz gráfica
        self.tile_size = 100
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.step_button = tk.Button(root, text="Ejecutar Softmax Q-Learning", command=self.run_softmax_q_learning)
        self.step_button.pack()

        self.reset_button = tk.Button(root, text="Reiniciar", command=self.reset)
        self.reset_button.pack(pady=10)

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona 'Ejecutar Softmax Q-Learning' para comenzar.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img'."""
        self.images = {}
        img_folder = "img"
        self.images["S"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["F"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["H"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["G"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["elf_down"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
        )

    def draw_map(self):
        """Dibuja el mapa del entorno."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images["elf_down"])

    def softmax(self, state):
        """Calcula las probabilidades de las acciones usando Softmax."""
        q_values = self.q_table[state]
        exp_values = np.exp(q_values / self.tau)
        probabilities = exp_values / np.sum(exp_values)
        return probabilities

    def select_action_softmax(self, state):
        """Selecciona una acción basada en las probabilidades Softmax."""
        probabilities = self.softmax(state)
        return np.random.choice(len(probabilities), p=probabilities)

    def run_softmax_q_learning(self):
        """Ejecuta el algoritmo Q-Learning con exploración Softmax."""
        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False
            step = 0

            while not done:
                # Seleccionar acción usando Softmax
                action = self.select_action_softmax(state_index)

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Actualizar la tabla Q usando la regla de Q-Learning
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                # Actualizar estado
                state_index = next_state_index
                self.state = next_state
                done = terminated
                step += 1

                # Redibujar el mapa
                self.draw_map()
                self.root.update()

                # Verificar si se alcanzó el objetivo
                if terminated and reward > 0:
                    self.success_count += 1

            # Reducir temperatura después de cada episodio
            self.tau = max(0.1, self.tau * 0.995)

            # Actualizar el historial de éxitos cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = self.success_count / 100
                self.success_history.append(success_rate)
                self.success_count = 0

        # Mostrar resultados finales
        #self.message_label.config(text=f"Softmax Q-Learning completado. Tasa de éxito: {self.success_history[-1]:.2f}")

    def reset(self):
        """Reinicia el entorno."""
        self.state = self.lake.reset()
        self.q_table = np.zeros((self.lake.n_rows * self.lake.n_cols, 4))
        self.current_episode = 0
        self.done = False
        self.success_count = 0
        self.success_history = []
        self.tau = 1.0  # Restaurar temperatura inicial
        self.draw_map()
        self.message_label.config(text="Presiona 'Ejecutar Softmax Q-Learning' para comenzar.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake - Ejercicio 4: Softmax Q-Learning")
app = SoftmaxQLearningVisualizer(root, lake, episodes=500, tau=1.0)
root.mainloop()


In [7]:
def evaluate_model(agent, lake, evaluation_episodes=100):
    """
    Evalúa el modelo entrenado siguiendo únicamente la política aprendida.
    
    Parámetros:
    - agent: Modelo entrenado con Q-Learning o SARSA.
    - lake: Entorno FrozenLake.
    - evaluation_episodes: Número de episodios de evaluación.
    
    Devuelve:
    - Tasa de éxito durante la evaluación.
    """
    success_count = 0

    for episode in range(evaluation_episodes):
        state = lake.reset()  # Reiniciar el entorno
        state_index = state[0] * lake.n_cols + state[1]  # Índice del estado
        done = False

        while not done:
            # Seleccionar la mejor acción según la tabla Q
            action = np.argmax(agent.q_table[state_index])

            # Realizar la acción en el entorno
            next_state, reward, terminated = lake.step(action)
            state_index = next_state[0] * lake.n_cols + next_state[1]
            done = terminated

            # Contar éxito si se alcanza la meta
            if reward > 0:
                success_count += 1

    success_rate = (success_count / evaluation_episodes) * 100
    print(f"Evaluación completada en {evaluation_episodes} episodios.")
    print(f"Tasa de éxito: {success_rate:.2f}%")
    return success_rate


# Configurar el entorno FrozenLake - Estocasticidad 
lake = FrozenLake(is_slippery=False)

# Entrenar al agente con Q-Learning
agent = QLearning(lake, episodes=6000)
agent.train()

# Evaluar el modelo entrenado
evaluate_model(agent, lake, evaluation_episodes=100)


Episodio 100/6000: Tasa de éxito: 81.00%
Episodio 200/6000: Tasa de éxito: 86.50%
Episodio 300/6000: Tasa de éxito: 90.33%
Episodio 400/6000: Tasa de éxito: 91.75%
Episodio 500/6000: Tasa de éxito: 93.00%
Episodio 600/6000: Tasa de éxito: 94.00%
Episodio 700/6000: Tasa de éxito: 94.71%
Episodio 800/6000: Tasa de éxito: 95.38%
Episodio 900/6000: Tasa de éxito: 95.89%
Episodio 1000/6000: Tasa de éxito: 96.00%
Episodio 1100/6000: Tasa de éxito: 96.18%
Episodio 1200/6000: Tasa de éxito: 96.33%
Episodio 1300/6000: Tasa de éxito: 96.62%
Episodio 1400/6000: Tasa de éxito: 96.71%
Episodio 1500/6000: Tasa de éxito: 96.93%
Episodio 1600/6000: Tasa de éxito: 97.12%
Episodio 1700/6000: Tasa de éxito: 97.18%
Episodio 1800/6000: Tasa de éxito: 97.22%
Episodio 1900/6000: Tasa de éxito: 97.32%
Episodio 2000/6000: Tasa de éxito: 97.45%
Episodio 2100/6000: Tasa de éxito: 97.48%
Episodio 2200/6000: Tasa de éxito: 97.55%
Episodio 2300/6000: Tasa de éxito: 97.61%
Episodio 2400/6000: Tasa de éxito: 97.71%
E

100.0

### **Ejercicio 5: Decaimiento de ε**
- **Objetivo**:
  - Diseñar un esquema para reducir $ \epsilon $ con el tiempo.
  - Evaluar cómo mejora la política aprendida en comparación con la política de exploración.


In [60]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import random


class DecayEpsilonQLearningVisualizer:
    def __init__(self, root, lake, episodes=500, epsilon_0=0.5, epsilon_min=0.01, decay=0.995):
        self.root = root
        self.lake = lake
        self.episodes = episodes
        self.epsilon = epsilon_0  # Valor inicial de epsilon
        self.epsilon_min = epsilon_min  # Valor mínimo de epsilon
        self.decay = decay  # Factor de decaimiento
        self.alpha = 0.5  # Tasa de aprendizaje
        self.gamma = 0.99  # Factor de descuento
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Tabla Q

        # Configuración del episodio actual
        self.current_episode = 0
        self.state = self.lake.reset()
        self.done = False
        self.success_count = 0
        self.success_history = []  # Historial de éxitos (en ventanas de 100 episodios)

        # Configuración de la interfaz gráfica
        self.tile_size = 100
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.step_button = tk.Button(root, text="Ejecutar Q-Learning con Decaimiento de ε", command=self.run_q_learning)
        self.step_button.pack()

        self.reset_button = tk.Button(root, text="Reiniciar", command=self.reset)
        self.reset_button.pack(pady=10)

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona 'Ejecutar Q-Learning con Decaimiento de ε' para comenzar.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img'."""
        self.images = {}
        img_folder = "img"
        self.images["S"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["F"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["H"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["G"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["elf_down"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
        )

    def draw_map(self):
        """Dibuja el mapa del entorno."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images["elf_down"])

    def epsilon_greedy(self, state):
        """Selecciona una acción usando ε-greedy."""
        if np.random.uniform(0, 1) < self.epsilon:
            return random.choice([0, 1, 2, 3])  # Exploración
        else:
            return np.argmax(self.q_table[state])  # Explotación

    def run_q_learning(self):
        """Ejecuta el algoritmo Q-Learning con decaimiento de ε."""
        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False
            step = 0

            while not done:
                # Seleccionar acción usando ε-greedy
                action = self.epsilon_greedy(state_index)

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Actualizar la tabla Q usando la regla de Q-Learning
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                # Actualizar estado
                state_index = next_state_index
                self.state = next_state
                done = terminated
                step += 1

                # Redibujar el mapa
                self.draw_map()
                self.root.update()

                # Verificar si se alcanzó el objetivo
                if terminated and reward > 0:
                    self.success_count += 1

            # Reducir ε después de cada episodio
            self.epsilon = max(self.epsilon_min, self.epsilon * self.decay)

            # Actualizar el historial de éxitos cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = self.success_count / 100
                self.success_history.append(success_rate)
                self.success_count = 0

        # Mostrar resultados finales
        self.message_label.config(text=f"Q-Learning completado. Tasa de éxito: {self.success_history[-1]:.2f}")

    def reset(self):
        """Reinicia el entorno."""
        self.state = self.lake.reset()
        self.q_table = np.zeros((self.lake.n_rows * self.lake.n_cols, 4))
        self.epsilon = 0.5  # Restaurar epsilon inicial
        self.current_episode = 0
        self.done = False
        self.success_count = 0
        self.success_history = []
        self.draw_map()
        self.message_label.config(text="Presiona 'Ejecutar Q-Learning con Decaimiento de ε' para comenzar.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake - Ejercicio 5: Q-Learning con Decaimiento de ε")
app = DecayEpsilonQLearningVisualizer(root, lake, episodes=500, epsilon_0=0.5, epsilon_min=0.01, decay=0.995)
root.mainloop()


In [ ]:
import numpy as np
import random


class QLearningWithDecay:
    def __init__(self, lake, episodes=5000, alpha=0.5, gamma=0.99, epsilon=1.0, decay=0.995):
        self.lake = lake
        self.episodes = episodes
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon  # Valor inicial de exploración
        self.epsilon_decay = decay  # Tasa de reducción de  epsilon 
        self.epsilon_min = 0.01  # Valor mínimo de epsilon
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))

    def epsilon_greedy(self, state_index):
        """Selecciona una acción usando ε-greedy."""
        if np.random.uniform(0, 1) < self.epsilon:
            return random.choice([0, 1, 2, 3])  # Exploración
        else:
            return np.argmax(self.q_table[state_index])  # Explotación

    def train(self):
        """Entrena al agente usando Q-Learning con decaimiento de epsilon ."""
        success_count = 0
        success_history = []

        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False

            while not done:
                action = self.epsilon_greedy(state_index)
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                state_index = next_state_index
                done = terminated

                if reward > 0:
                    success_count += 1

            # Reducir epsilon despues de cada episodio
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            if (episode + 1) % 100 == 0:
                success_rate = (success_count / (episode + 1)) * 100
                success_history.append(success_rate)
                print(f"Episodio {episode + 1}/{self.episodes}: Tasa de éxito: {success_rate:.2f}%")

        print("Entrenamiento completado.")
        print(f"Tasa de éxito final: {success_rate:.2f}%")
        return success_history

    def evaluate(self, evaluation_episodes=100):
        """Evalúa al agente siguiendo únicamente la política greedy."""
        success_count = 0

        for _ in range(evaluation_episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False

            while not done:
                action = np.argmax(self.q_table[state_index])  # Política greedy
                next_state, reward, terminated = self.lake.step(action)
                state_index = next_state[0] * self.lake.n_cols + next_state[1]
                done = terminated

                if reward > 0:
                    success_count += 1

        success_rate = (success_count / evaluation_episodes) * 100
        print(f"Evaluación completada. Tasa de éxito: {success_rate:.2f}%")
        return success_rate


# Configurar el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Entrenar al agente con decaimiento de epsilon
agent = QLearningWithDecay(lake, episodes=5000, epsilon=1.0, decay=0.995)
agent.train()

# Evaluar el modelo
agent.evaluate(evaluation_episodes=100)


Episodio 100/5000: Tasa de éxito: 7.00%
Episodio 200/5000: Tasa de éxito: 29.00%
Episodio 300/5000: Tasa de éxito: 42.67%
Episodio 400/5000: Tasa de éxito: 52.00%
Episodio 500/5000: Tasa de éxito: 59.20%
Episodio 600/5000: Tasa de éxito: 65.33%
Episodio 700/5000: Tasa de éxito: 70.00%
Episodio 800/5000: Tasa de éxito: 73.38%
Episodio 900/5000: Tasa de éxito: 76.11%
Episodio 1000/5000: Tasa de éxito: 78.50%
Episodio 1100/5000: Tasa de éxito: 80.18%
Episodio 1200/5000: Tasa de éxito: 81.75%
Episodio 1300/5000: Tasa de éxito: 83.08%
Episodio 1400/5000: Tasa de éxito: 84.29%
Episodio 1500/5000: Tasa de éxito: 85.33%
Episodio 1600/5000: Tasa de éxito: 86.25%
Episodio 1700/5000: Tasa de éxito: 87.06%
Episodio 1800/5000: Tasa de éxito: 87.67%
Episodio 1900/5000: Tasa de éxito: 88.26%
Episodio 2000/5000: Tasa de éxito: 88.85%
Episodio 2100/5000: Tasa de éxito: 89.38%
Episodio 2200/5000: Tasa de éxito: 89.77%
Episodio 2300/5000: Tasa de éxito: 90.13%
Episodio 2400/5000: Tasa de éxito: 90.50%
Ep

100.0

### **Ejercicio 6: Evaluación Apropiada**
- **Objetivo**:
  - Implementar un bucle interno para evaluar las políticas aprendidas repetidamente.
  - Comparar el rendimiento promedio entre SARSA y Q-Learning.
  - Concluir en qué circunstancias un algoritmo es mejor que el otro.



In [96]:
def run_experiments(lake, configs, episodes=5000, evaluation_episodes=100):
    """
    Ejecuta experimentos con diferentes configuraciones de hiperparámetros.
    
    Parámetros:
    - lake: Entorno FrozenLake.
    - configs: Lista de configuraciones de hiperparámetros.
    - episodes: Número de episodios para entrenamiento.
    - evaluation_episodes: Número de episodios para evaluación.
    
    Devuelve:
    - Resultados de cada configuración.
    """
    results = []

    for config in configs:
        print(f"Entrenando con configuración: α={config['alpha']}, γ={config['gamma']}, "
              f"ε={config['epsilon']}, decay={config['epsilon_decay']}")

        # Crear el agente con la configuración actual
        agent = QLearningWithDecay(
            lake=lake,
            episodes=episodes,
            alpha=config['alpha'],
            gamma=config['gamma'],
            epsilon=config['epsilon'],
            decay=config['epsilon_decay']
        )
        
        # Entrenar y evaluar
        agent.train()
        success_rate = agent.evaluate(evaluation_episodes=evaluation_episodes)
        
        # Guardar resultados
        results.append({
            "alpha": config["alpha"],
            "gamma": config["gamma"],
            "epsilon": config["epsilon"],
            "epsilon_decay": config["epsilon_decay"],
            "success_rate": success_rate
        })

    return results


# Configuraciones de Hiperparámetros
configs = [
    {"alpha": 0.1, "gamma": 0.9, "epsilon": 1.0, "epsilon_decay": 0.995},
    {"alpha": 0.5, "gamma": 0.99, "epsilon": 0.5, "epsilon_decay": 0.995},
    {"alpha": 0.7, "gamma": 0.8, "epsilon": 0.3, "epsilon_decay": 0.9},
    {"alpha": 0.3, "gamma": 0.95, "epsilon": 1.0, "epsilon_decay": 0.99},
]

# Configurar el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Ejecutar los experimentos
results = run_experiments(lake, configs)

# Imprimir resultados
for res in results:
    print(f"Configuración: α={res['alpha']}, γ={res['gamma']}, ε={res['epsilon']}, decay={res['epsilon_decay']}")
    print(f"Tasa de éxito: {res['success_rate']:.2f}%\n")


Entrenando con configuración: α=0.1, γ=0.9, ε=1.0, decay=0.995
Episodio 100/5000: Tasa de éxito: 3.00%
Episodio 200/5000: Tasa de éxito: 2.00%
Episodio 300/5000: Tasa de éxito: 4.00%
Episodio 400/5000: Tasa de éxito: 6.25%
Episodio 500/5000: Tasa de éxito: 8.20%
Episodio 600/5000: Tasa de éxito: 9.17%
Episodio 700/5000: Tasa de éxito: 11.29%
Episodio 800/5000: Tasa de éxito: 15.25%
Episodio 900/5000: Tasa de éxito: 17.89%
Episodio 1000/5000: Tasa de éxito: 20.80%
Episodio 1100/5000: Tasa de éxito: 23.18%
Episodio 1200/5000: Tasa de éxito: 25.83%
Episodio 1300/5000: Tasa de éxito: 28.00%
Episodio 1400/5000: Tasa de éxito: 29.00%
Episodio 1500/5000: Tasa de éxito: 30.40%
Episodio 1600/5000: Tasa de éxito: 32.38%
Episodio 1700/5000: Tasa de éxito: 33.29%
Episodio 1800/5000: Tasa de éxito: 33.94%
Episodio 1900/5000: Tasa de éxito: 34.74%
Episodio 2000/5000: Tasa de éxito: 34.85%
Episodio 2100/5000: Tasa de éxito: 35.86%
Episodio 2200/5000: Tasa de éxito: 37.68%
Episodio 2300/5000: Tasa de 